In [46]:
import requests
from bs4 import BeautifulSoup
import json
import time
import csv

In [7]:
url = 'https://www.imdb.com/chart/boxoffice'
html = requests.get(url).text
soup = BeautifulSoup(html, 'html.parser')

In [20]:
soup.find('div', {'id': 'boxoffice'}).h4.text

'Weekend of May 10 - 12, 2019'

In [17]:
soup.find('div', {'id': 'boxoffice'}).table.findAll('tr')[1:][0]

<tr>
<td class="posterColumn">
<a href="/title/tt4154796"> <img alt="Avengers: Endgame" height="67" src="https://m.media-amazon.com/images/M/MV5BMTc5MDE2ODcwNV5BMl5BanBnXkFtZTgwMzI2NzQ2NzM@._V1_UY67_CR0,0,45,67_AL_.jpg" width="45"/>
</a> <td class="titleColumn">
<a href="/title/tt4154796" title="Anthony Russo (dir.), Robert Downey Jr., Chris Evans">Avengers: Endgame</a>
</td>
<td class="ratingColumn">
                            $63.3M
                    </td>
<td class="ratingColumn">
<span class="secondaryInfo">$723.7M</span>
</td>
<td class="weeksColumn">3</td>
<td class="watchlistColumn"><div class="wlb_ribbon" data-tconst="tt4154796"></div></td>
</td></tr>

In [49]:
start = time.time()
base_imdb_url = 'https://www.imdb.com'
html = requests.get(base_imdb_url + '/chart/boxoffice').text
soup = BeautifulSoup(html, 'html.parser')

boxoffice = {}
boxoffice['date'] = soup.find('div', {'id': 'boxoffice'}).h4.text
boxoffice['movies'] = []

movie_list = []
for movie in soup.find('div', {'id': 'boxoffice'}).table.findAll('tr')[1:]:
    movie_list.append(movie.find('td', class_='titleColumn').a.text)

fieldnames = ['Year', 'Rated', 'Released', 'Runtime', 'Genre', 'Director', 'Actors', 'Plot',
              'Poster', 'imdbID', 'Production']
for idx, title in enumerate(movie_list):
    temp = {'Country': 'us', 'Rank': str(idx+1), 'Title': title}
    url = f'http://www.omdbapi.com/?apikey=556ed6ed&t={title}&y=2019&plot=full'
    data = json.loads(requests.get(url).text)
    if data['Response'] == 'True':
        for field in fieldnames:
            temp[field] = data[field]
    
        imdbID = temp['imdbID']
        html = requests.get(base_imdb_url + f'/title/{imdbID}').text
        soup = BeautifulSoup(html, 'html.parser')
        temp['Trailer'] = base_imdb_url + soup.find('div', class_='slate').find('a')['href']
        temp['Trailer_img'] = soup.find('div', class_='slate').find('img')['src']
    
    boxoffice['movies'].append(temp)

print(time.time()-start)

16.923370122909546


In [45]:
boxoffice

{'date': 'Weekend of May 10 - 12, 2019',
 'movies': [{'Rank': '1',
   'Title': 'Avengers: Endgame',
   'Year': '2019',
   'Rated': 'PG-13',
   'Released': '26 Apr 2019',
   'Runtime': '181 min',
   'Genre': 'Action, Adventure, Fantasy, Sci-Fi',
   'Director': 'Anthony Russo, Joe Russo',
   'Actors': 'Robert Downey Jr., Chris Evans, Mark Ruffalo, Chris Hemsworth',
   'Plot': "After the devastating events of Avengers: Infinity War (2018), the universe is in ruins. With the help of remaining allies, the Avengers assemble once more in order to undo Thanos' actions and restore order to the universe.",
   'Poster': 'https://m.media-amazon.com/images/M/MV5BMTc5MDE2ODcwNV5BMl5BanBnXkFtZTgwMzI2NzQ2NzM@._V1_SX300.jpg',
   'imdbID': 'tt4154796',
   'Production': 'Marvel Studios',
   'Trailer': 'https://www.imdb.com/video/imdb/vi2163260441?playlistId=tt4154796',
   'Trailer_img': 'https://m.media-amazon.com/images/M/MV5BOThlNjdhZmQtNzlkOS00M2VjLWI0ZjUtZDExZDI1MjRhZGFkXkEyXkFqcGdeQW1yb3NzZXI@._V1_U

In [51]:
with open('us.csv', 'w') as f:
    fieldnames = ['Country', 'Rank', 'Title', 'Year', 'Rated', 'Released', 'Runtime', 'Genre', 'Director',
                  'Actors', 'Plot', 'Poster', 'imdbID', 'Production', 'Trailer', 'Trailer_img']
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    for movie in boxoffice['movies']:
        writer.writerow(movie)

In [35]:
movie = boxoffice['movies'][0]
imdbID = movie['imdbID']
url = f'https://www.imdb.com/title/{imdbID}/'
html = requests.get(url).text
soup = BeautifulSoup(html, 'html.parser')

In [36]:
soup


<!DOCTYPE html>

<html xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://ogp.me/ns#">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="app-id=342792525, app-argument=imdb:///title/tt4154796?src=mdot" name="apple-itunes-app"/>
<script type="text/javascript">var IMDbTimer={starttime: new Date().getTime(),pt:'java'};</script>
<script>
    if (typeof uet == 'function') {
      uet("bb", "LoadTitle", {wb: 1});
    }
</script>
<script>(function(t){ (t.events = t.events || {})["csm_head_pre_title"] = new Date().getTime(); })(IMDbTimer);</script>
<title>Avengers: Endgame (2019) - IMDb</title>
<script>(function(t){ (t.events = t.events || {})["csm_head_post_title"] = new Date().getTime(); })(IMDbTimer);</script>
<script>
    if (typeof uet == 'function') {
      uet("be", "LoadTitle", {wb: 1});
    }
</script>
<script>
    if (typeof uex == 'function') {
      uex("ld", "LoadTitle", {wb: 1});
    }
</script>
<link href="https://ww

In [43]:
trailer = soup.find('div', class_='slate').find('a')['href']
trailer_img = soup.find('div', class_='slate').find('img')['src']

'/video/imdb/vi2163260441?playlistId=tt4154796'